#Preface





This document is a report written for the course in "Blockchain Engineering" taken at 
Delft University of Technology in 2018. It was written to aid the Blockchain Lab's
research on cooperation in distributed systems. 

In it we discuss the implementation of 
an algorithm to determine the trustworthiness of agents in a peer-to-peer file sharing 
network, called Tribler. 

In this way, we hope to contribute to the lab's long-term goal of achieving a universal
mechanism of generating trust in distributed networks.


 


Hello
